In [1]:
for i in range(0, 701, 7):
    print(i)

0
7
14
21
28
35
42
49
56
63
70
77
84
91
98
105
112
119
126
133
140
147
154
161
168
175
182
189
196
203
210
217
224
231
238
245
252
259
266
273
280
287
294
301
308
315
322
329
336
343
350
357
364
371
378
385
392
399
406
413
420
427
434
441
448
455
462
469
476
483
490
497
504
511
518
525
532
539
546
553
560
567
574
581
588
595
602
609
616
623
630
637
644
651
658
665
672
679
686
693
700


In [ ]:
import pygame
import random
import math
import cv2
import numpy as np

pygame.init()

WIDTH, HEIGHT = 800, 600
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Непересекающиеся летающие фигуры")

WHITE = (255, 255, 255)

NUM_SHAPES = 7
MARGIN = 50

SPEED_MIN = 80
SPEED_MAX = 140

SAVE_VIDEO = True
VIDEO_PATH = "flying_shapes.mp4"
VIDEO_FPS = 60

TIME_LIMIT = 60.0  # секунды


class FlyingShape:
    def __init__(self, band_index, total_bands):
        self.band_index = band_index
        self.total_bands = total_bands
        self.reset()

    def reset(self):
        band_height = HEIGHT / self.total_bands
        self.y_min = self.band_index * band_height
        self.y_max = (self.band_index + 1) * band_height
        band_mid = 0.5 * (self.y_min + self.y_max)

        max_size = int(band_height / 3)
        min_size = 15
        if max_size < min_size:
            max_size = min_size
        self.size = random.randint(min_size, max_size)

        self.color = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255),
        )
        self.shape_type = random.choice(["circle", "rect", "triangle"])

        self.direction = random.choice(["left_to_right", "right_to_left"])
        self.speed = random.uniform(SPEED_MIN, SPEED_MAX)

        free_half_height = band_height / 2 - self.size - 4
        if free_half_height < 0:
            free_half_height = 0
        self.amplitude = random.uniform(0, free_half_height)
        self.frequency = random.uniform(0.5, 1.5)
        self.phase = random.uniform(0, 2 * math.pi)

        if self.direction == "left_to_right":
            self.x = -MARGIN - self.size
        else:
            self.x = WIDTH + MARGIN + self.size

        self.band_mid = band_mid
        self.y = band_mid

    def update(self, dt):
        if self.direction == "left_to_right":
            self.x += self.speed * dt
        else:
            self.x -= self.speed * dt

        if self.direction == "left_to_right":
            norm_x = (self.x + MARGIN) / (WIDTH + 2 * MARGIN)
        else:
            norm_x = (WIDTH + MARGIN - self.x) / (WIDTH + 2 * MARGIN)

        angle = 2 * math.pi * self.frequency * norm_x + self.phase
        self.y = self.band_mid + self.amplitude * math.sin(angle)

        if self.direction == "left_to_right" and self.x > WIDTH + MARGIN + self.size:
            self.reset()
        elif self.direction == "right_to_left" and self.x < -MARGIN - self.size:
            self.reset()

    def draw(self, surface):
        cx, cy = int(self.x), int(self.y)
        s = self.size

        if self.shape_type == "circle":
            pygame.draw.circle(surface, self.color, (cx, cy), s)
        elif self.shape_type == "rect":
            rect = pygame.Rect(cx - s, cy - s, 2 * s, 2 * s)
            pygame.draw.rect(surface, self.color, rect)
        elif self.shape_type == "triangle":
            p1 = (cx, cy - s)
            p2 = (cx - s, cy + s)
            p3 = (cx + s, cy + s)
            pygame.draw.polygon(surface, self.color, [p1, p2, p3])


def main():
    clock = pygame.time.Clock()
    running = True

    shapes = [FlyingShape(i, NUM_SHAPES) for i in range(NUM_SHAPES)]

    video_writer = None
    if SAVE_VIDEO:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        video_writer = cv2.VideoWriter(VIDEO_PATH, fourcc, VIDEO_FPS, (WIDTH, HEIGHT))

    start_ticks = pygame.time.get_ticks()

    while running:
        dt = clock.tick(60) / 1000.0

        # автостоп по таймеру
        elapsed = (pygame.time.get_ticks() - start_ticks) / 1000.0
        if elapsed >= TIME_LIMIT:
            running = False

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                running = False

        SCREEN.fill(WHITE)

        for shape in shapes:
            shape.update(dt)
            shape.draw(SCREEN)

        if SAVE_VIDEO and video_writer is not None:
            frame = pygame.surfarray.array3d(SCREEN)  # (W, H, 3)
            frame = np.transpose(frame, (1, 0, 2))    # (H, W, 3)
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            video_writer.write(frame_bgr)

        pygame.display.flip()

    if video_writer is not None:
        video_writer.release()

    pygame.quit()


if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
